In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib

In [5]:
cust_df = pd.read_csv(
        '../data/scs/train.csv' #데이터 경로
        ,encoding='latin-1' #인코딩 : 영어
)
print(cust_df.shape) #(76020, 371) = (고객수, 컬럼수)
cust_df.head(3) #Target은 답에 대한 lable, 그 이외가 실질적 데이터

(76020, 371)


,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.17,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.03,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.77,0


In [8]:
#데이터와 답 분류하기
# 피처 세트와 레이블 세트분리. 레이블 컬럼은 DataFrame의 맨 마지막에 위치해 컬럼 위치 -1로 분리
X_features = cust_df.iloc[:, :-1]
y_labels = cust_df.iloc[:, -1]
print('피처 데이터 shape:{0}'.format(X_features.shape))

피처 데이터 shape:(76020, 370)


In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_features, y_labels,
                                                    test_size=0.2, random_state=0)
train_cnt = y_train.count()
test_cnt = y_test.count()
print('학습 세트 Shape:{0}, 테스트 세트 Shape:{1}'.format(X_train.shape , X_test.shape))

학습 세트 Shape:(60816, 370), 테스트 세트 Shape:(15204, 370)


# 3. LightGBM 적용

In [6]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

In [10]:
# n_estimators는 1000으로 증가시키고, learning_rate=0.02로 감소, reg_alpha=0.03으로 추가함. 
xgb_clf = XGBClassifier(n_estimators=1000, random_state=156, learning_rate=0.02, max_depth=5,\
                        min_child_weight=1, colsample_bytree=0.75, reg_alpha=0.03)

# evaluation metric을 auc로, early stopping은 200 으로 설정하고 학습 수행. 
xgb_clf.fit(X_train, y_train, early_stopping_rounds=200, 
            eval_metric="auc",eval_set=[(X_train, y_train), (X_test, y_test)])

xgb_roc_score = roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:,1],average='macro')
print('ROC AUC: {0:.4f}'.format(xgb_roc_score))

[0]	validation_0-auc:0.817927	validation_1-auc:0.812652
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 200 rounds.
[1]	validation_0-auc:0.820523	validation_1-auc:0.812985
[2]	validation_0-auc:0.821277	validation_1-auc:0.813444
[3]	validation_0-auc:0.821432	validation_1-auc:0.813236
[4]	validation_0-auc:0.824602	validation_1-auc:0.815378
[5]	validation_0-auc:0.825073	validation_1-auc:0.816048
[6]	validation_0-auc:0.830171	validation_1-auc:0.820443
[7]	validation_0-auc:0.829757	validation_1-auc:0.819594
[8]	validation_0-auc:0.829691	validation_1-auc:0.819712
[9]	validation_0-auc:0.82977	validation_1-auc:0.819963
[10]	validation_0-auc:0.830901	validation_1-auc:0.821368
[11]	validation_0-auc:0.830908	validation_1-auc:0.821209
[12]	validation_0-auc:0.830919	validation_1-auc:0.822656
[13]	validation_0-auc:0.831363	validation_1-auc:0.823147
[14]	validation_0-auc:0.833543	validation_1-auc:0.82654

[141]	validation_0-auc:0.856497	validation_1-auc:0.840529
[142]	validation_0-auc:0.856602	validation_1-auc:0.84072
[143]	validation_0-auc:0.856709	validation_1-auc:0.840827
[144]	validation_0-auc:0.856857	validation_1-auc:0.841015
[145]	validation_0-auc:0.856883	validation_1-auc:0.841067
[146]	validation_0-auc:0.856901	validation_1-auc:0.8411
[147]	validation_0-auc:0.85703	validation_1-auc:0.841192
[148]	validation_0-auc:0.857166	validation_1-auc:0.841308
[149]	validation_0-auc:0.857273	validation_1-auc:0.841236
[150]	validation_0-auc:0.857472	validation_1-auc:0.841365
[151]	validation_0-auc:0.857458	validation_1-auc:0.841362
[152]	validation_0-auc:0.857651	validation_1-auc:0.841406
[153]	validation_0-auc:0.857913	validation_1-auc:0.84139
[154]	validation_0-auc:0.85794	validation_1-auc:0.84152
[155]	validation_0-auc:0.858122	validation_1-auc:0.841485
[156]	validation_0-auc:0.858328	validation_1-auc:0.841686
[157]	validation_0-auc:0.858489	validation_1-auc:0.841665
[158]	validation_0-au

[283]	validation_0-auc:0.874082	validation_1-auc:0.843663
[284]	validation_0-auc:0.87419	validation_1-auc:0.843646
[285]	validation_0-auc:0.874275	validation_1-auc:0.84375
[286]	validation_0-auc:0.874336	validation_1-auc:0.843735
[287]	validation_0-auc:0.874471	validation_1-auc:0.843676
[288]	validation_0-auc:0.874552	validation_1-auc:0.84367
[289]	validation_0-auc:0.874661	validation_1-auc:0.84364
[290]	validation_0-auc:0.874733	validation_1-auc:0.843652
[291]	validation_0-auc:0.874831	validation_1-auc:0.843668
[292]	validation_0-auc:0.874945	validation_1-auc:0.843607
[293]	validation_0-auc:0.875025	validation_1-auc:0.843587
[294]	validation_0-auc:0.875105	validation_1-auc:0.843623
[295]	validation_0-auc:0.8752	validation_1-auc:0.843597
[296]	validation_0-auc:0.875285	validation_1-auc:0.843622
[297]	validation_0-auc:0.875432	validation_1-auc:0.843737
[298]	validation_0-auc:0.875588	validation_1-auc:0.843761
[299]	validation_0-auc:0.875662	validation_1-auc:0.843714
[300]	validation_0-a

[425]	validation_0-auc:0.884422	validation_1-auc:0.843473
Stopping. Best iteration:
[225]	validation_0-auc:0.868032	validation_1-auc:0.844342

ROC AUC: 0.8443


In [11]:
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier(n_estimators=500)

evals = [(X_test, y_test)]
lgbm_clf.fit(X_train, y_train, early_stopping_rounds=100, eval_metric="auc", eval_set=evals,
                verbose=True)

lgbm_roc_score = roc_auc_score(y_test, lgbm_clf.predict_proba(X_test)[:,1],average='macro')
print('ROC AUC: {0:.4f}'.format(lgbm_roc_score))

[1]	valid_0's auc: 0.817877	valid_0's binary_logloss: 0.165052
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 0.817879	valid_0's binary_logloss: 0.160075
[3]	valid_0's auc: 0.82723	valid_0's binary_logloss: 0.156418
[4]	valid_0's auc: 0.831084	valid_0's binary_logloss: 0.153598
[5]	valid_0's auc: 0.833542	valid_0's binary_logloss: 0.151396
[6]	valid_0's auc: 0.833045	valid_0's binary_logloss: 0.149583
[7]	valid_0's auc: 0.834596	valid_0's binary_logloss: 0.148134
[8]	valid_0's auc: 0.836232	valid_0's binary_logloss: 0.146835
[9]	valid_0's auc: 0.837247	valid_0's binary_logloss: 0.145888
[10]	valid_0's auc: 0.838312	valid_0's binary_logloss: 0.144884
[11]	valid_0's auc: 0.839099	valid_0's binary_logloss: 0.144052
[12]	valid_0's auc: 0.83924	valid_0's binary_logloss: 0.143418
[13]	valid_0's auc: 0.83934	valid_0's binary_logloss: 0.142902
[14]	valid_0's auc: 0.839012	valid_0's binary_logloss: 0.142465
[15]	valid_0's auc: 0.839228	valid_0's binary_logloss:

In [12]:
from sklearn.model_selection import GridSearchCV

# 하이퍼 파라미터 테스트의 수행 속도를 향상시키기 위해 n_estimators를 100으로 감소
LGBM_clf = LGBMClassifier(n_estimators=200)

params = {'num_leaves': [32, 64 ],
          'max_depth':[128, 160],
          'min_child_samples':[60, 100],
          'subsample':[0.8, 1]}


# 하이퍼 파라미터 테스트의 수행속도를 향상 시키기 위해 cv 를 지정하지 않습니다. 
gridcv = GridSearchCV(lgbm_clf, param_grid=params)
gridcv.fit(X_train, y_train, early_stopping_rounds=30, eval_metric="auc",
           eval_set=[(X_train, y_train), (X_test, y_test)])

print('GridSearchCV 최적 파라미터:', gridcv.best_params_)
lgbm_roc_score = roc_auc_score(y_test, gridcv.predict_proba(X_test)[:,1], average='macro')
print('ROC AUC: {0:.4f}'.format(lgbm_roc_score))

[1]	valid_0's auc: 0.822699	valid_0's binary_logloss: 0.156165	valid_1's auc: 0.819684	valid_1's binary_logloss: 0.164905
Training until validation scores don't improve for 30 rounds
[2]	valid_0's auc: 0.829006	valid_0's binary_logloss: 0.150877	valid_1's auc: 0.823237	valid_1's binary_logloss: 0.159734
[3]	valid_0's auc: 0.833072	valid_0's binary_logloss: 0.14711	valid_1's auc: 0.823439	valid_1's binary_logloss: 0.156105
[4]	valid_0's auc: 0.838913	valid_0's binary_logloss: 0.144118	valid_1's auc: 0.828353	valid_1's binary_logloss: 0.153339
[5]	valid_0's auc: 0.842187	valid_0's binary_logloss: 0.141618	valid_1's auc: 0.83212	valid_1's binary_logloss: 0.151007
[6]	valid_0's auc: 0.845068	valid_0's binary_logloss: 0.13954	valid_1's auc: 0.834388	valid_1's binary_logloss: 0.149179
[7]	valid_0's auc: 0.846442	valid_0's binary_logloss: 0.137857	valid_1's auc: 0.834683	valid_1's binary_logloss: 0.147638
[8]	valid_0's auc: 0.849072	valid_0's binary_logloss: 0.136401	valid_1's auc: 0.836625	v

[29]	valid_0's auc: 0.878752	valid_0's binary_logloss: 0.123469	valid_1's auc: 0.836869	valid_1's binary_logloss: 0.140112
[30]	valid_0's auc: 0.879985	valid_0's binary_logloss: 0.123104	valid_1's auc: 0.836812	valid_1's binary_logloss: 0.140074
[31]	valid_0's auc: 0.880668	valid_0's binary_logloss: 0.122813	valid_1's auc: 0.836283	valid_1's binary_logloss: 0.140118
[32]	valid_0's auc: 0.881805	valid_0's binary_logloss: 0.122494	valid_1's auc: 0.836142	valid_1's binary_logloss: 0.140076
[33]	valid_0's auc: 0.882473	valid_0's binary_logloss: 0.122227	valid_1's auc: 0.835876	valid_1's binary_logloss: 0.140107
[34]	valid_0's auc: 0.883187	valid_0's binary_logloss: 0.121948	valid_1's auc: 0.835659	valid_1's binary_logloss: 0.140116
[35]	valid_0's auc: 0.884136	valid_0's binary_logloss: 0.121665	valid_1's auc: 0.835638	valid_1's binary_logloss: 0.140073
[36]	valid_0's auc: 0.88504	valid_0's binary_logloss: 0.121365	valid_1's auc: 0.835398	valid_1's binary_logloss: 0.140086
[37]	valid_0's au

[55]	valid_0's auc: 0.896603	valid_0's binary_logloss: 0.116667	valid_1's auc: 0.838375	valid_1's binary_logloss: 0.139428
[56]	valid_0's auc: 0.897008	valid_0's binary_logloss: 0.116479	valid_1's auc: 0.838435	valid_1's binary_logloss: 0.139435
[57]	valid_0's auc: 0.897488	valid_0's binary_logloss: 0.11627	valid_1's auc: 0.838521	valid_1's binary_logloss: 0.139409
[58]	valid_0's auc: 0.897929	valid_0's binary_logloss: 0.116102	valid_1's auc: 0.838294	valid_1's binary_logloss: 0.139474
[59]	valid_0's auc: 0.898419	valid_0's binary_logloss: 0.115908	valid_1's auc: 0.838608	valid_1's binary_logloss: 0.139435
Early stopping, best iteration is:
[29]	valid_0's auc: 0.877624	valid_0's binary_logloss: 0.123206	valid_1's auc: 0.841241	valid_1's binary_logloss: 0.13903
[1]	valid_0's auc: 0.822357	valid_0's binary_logloss: 0.156061	valid_1's auc: 0.820544	valid_1's binary_logloss: 0.164878
Training until validation scores don't improve for 30 rounds
[2]	valid_0's auc: 0.826115	valid_0's binary_l

[15]	valid_0's auc: 0.862843	valid_0's binary_logloss: 0.129813	valid_1's auc: 0.83881	valid_1's binary_logloss: 0.141988
[16]	valid_0's auc: 0.864132	valid_0's binary_logloss: 0.129175	valid_1's auc: 0.838261	valid_1's binary_logloss: 0.141767
[17]	valid_0's auc: 0.865507	valid_0's binary_logloss: 0.128552	valid_1's auc: 0.838112	valid_1's binary_logloss: 0.141529
[18]	valid_0's auc: 0.867478	valid_0's binary_logloss: 0.127927	valid_1's auc: 0.838026	valid_1's binary_logloss: 0.141312
[19]	valid_0's auc: 0.868762	valid_0's binary_logloss: 0.127396	valid_1's auc: 0.837868	valid_1's binary_logloss: 0.141082
[20]	valid_0's auc: 0.869852	valid_0's binary_logloss: 0.12688	valid_1's auc: 0.838527	valid_1's binary_logloss: 0.140842
[21]	valid_0's auc: 0.870916	valid_0's binary_logloss: 0.126436	valid_1's auc: 0.83881	valid_1's binary_logloss: 0.140636
[22]	valid_0's auc: 0.872234	valid_0's binary_logloss: 0.125974	valid_1's auc: 0.838623	valid_1's binary_logloss: 0.140492
[23]	valid_0's auc:

[41]	valid_0's auc: 0.888385	valid_0's binary_logloss: 0.119936	valid_1's auc: 0.836779	valid_1's binary_logloss: 0.139483
[42]	valid_0's auc: 0.889233	valid_0's binary_logloss: 0.119743	valid_1's auc: 0.836861	valid_1's binary_logloss: 0.139444
Early stopping, best iteration is:
[12]	valid_0's auc: 0.857789	valid_0's binary_logloss: 0.131861	valid_1's auc: 0.839266	valid_1's binary_logloss: 0.142969
[1]	valid_0's auc: 0.821198	valid_0's binary_logloss: 0.156276	valid_1's auc: 0.816442	valid_1's binary_logloss: 0.165109
Training until validation scores don't improve for 30 rounds
[2]	valid_0's auc: 0.828172	valid_0's binary_logloss: 0.150979	valid_1's auc: 0.819928	valid_1's binary_logloss: 0.160006
[3]	valid_0's auc: 0.836865	valid_0's binary_logloss: 0.147117	valid_1's auc: 0.828439	valid_1's binary_logloss: 0.15651
[4]	valid_0's auc: 0.841914	valid_0's binary_logloss: 0.144116	valid_1's auc: 0.831691	valid_1's binary_logloss: 0.153607
[5]	valid_0's auc: 0.844672	valid_0's binary_log

...

[13]	valid_0's auc: 0.877072	valid_0's binary_logloss: 0.127675	valid_1's auc: 0.838154	valid_1's binary_logloss: 0.142217
[14]	valid_0's auc: 0.878739	valid_0's binary_logloss: 0.126748	valid_1's auc: 0.837654	valid_1's binary_logloss: 0.141853
[15]	valid_0's auc: 0.880591	valid_0's binary_logloss: 0.125818	valid_1's auc: 0.837661	valid_1's binary_logloss: 0.141367
[16]	valid_0's auc: 0.881915	valid_0's binary_logloss: 0.125027	valid_1's auc: 0.836986	valid_1's binary_logloss: 0.141199
[17]	valid_0's auc: 0.883195	valid_0's binary_logloss: 0.12425	valid_1's auc: 0.836787	valid_1's binary_logloss: 0.140962
[18]	valid_0's auc: 0.885491	valid_0's binary_logloss: 0.123434	valid_1's auc: 0.836738	valid_1's binary_logloss: 0.140693
[19]	valid_0's auc: 0.886726	valid_0's binary_logloss: 0.122713	valid_1's auc: 0.836636	valid_1's binary_logloss: 0.140564
[20]	valid_0's auc: 0.888242	valid_0's binary_logloss: 0.122056	valid_1's auc: 0.837407	valid_1's binary_logloss: 0.140299
[21]	valid_0's au

[1]	valid_0's auc: 0.833073	valid_0's binary_logloss: 0.155714	valid_1's auc: 0.8135	valid_1's binary_logloss: 0.165293
Training until validation scores don't improve for 30 rounds
[2]	valid_0's auc: 0.843468	valid_0's binary_logloss: 0.149953	valid_1's auc: 0.824536	valid_1's binary_logloss: 0.159864
[3]	valid_0's auc: 0.84816	valid_0's binary_logloss: 0.145786	valid_1's auc: 0.828936	valid_1's binary_logloss: 0.156383
[4]	valid_0's auc: 0.850873	valid_0's binary_logloss: 0.142532	valid_1's auc: 0.831042	valid_1's binary_logloss: 0.153507
[5]	valid_0's auc: 0.857549	valid_0's binary_logloss: 0.139787	valid_1's auc: 0.834663	valid_1's binary_logloss: 0.151386
[6]	valid_0's auc: 0.860917	valid_0's binary_logloss: 0.137506	valid_1's auc: 0.83515	valid_1's binary_logloss: 0.149561
[7]	valid_0's auc: 0.863075	valid_0's binary_logloss: 0.135588	valid_1's auc: 0.836172	valid_1's binary_logloss: 0.147983
[8]	valid_0's auc: 0.865387	valid_0's binary_logloss: 0.133941	valid_1's auc: 0.836232	va

[29]	valid_0's auc: 0.89693	valid_0's binary_logloss: 0.117428	valid_1's auc: 0.833802	valid_1's binary_logloss: 0.140324
[30]	valid_0's auc: 0.898236	valid_0's binary_logloss: 0.116983	valid_1's auc: 0.833651	valid_1's binary_logloss: 0.140328
[31]	valid_0's auc: 0.899515	valid_0's binary_logloss: 0.11649	valid_1's auc: 0.833679	valid_1's binary_logloss: 0.1403
[32]	valid_0's auc: 0.900965	valid_0's binary_logloss: 0.116016	valid_1's auc: 0.833617	valid_1's binary_logloss: 0.140299
[33]	valid_0's auc: 0.902142	valid_0's binary_logloss: 0.115506	valid_1's auc: 0.834116	valid_1's binary_logloss: 0.140211
[34]	valid_0's auc: 0.903318	valid_0's binary_logloss: 0.115068	valid_1's auc: 0.833884	valid_1's binary_logloss: 0.140264
[35]	valid_0's auc: 0.904371	valid_0's binary_logloss: 0.114634	valid_1's auc: 0.833782	valid_1's binary_logloss: 0.140308
[36]	valid_0's auc: 0.905453	valid_0's binary_logloss: 0.114293	valid_1's auc: 0.83325	valid_1's binary_logloss: 0.140417
[37]	valid_0's auc: 0

[7]	valid_0's auc: 0.863058	valid_0's binary_logloss: 0.135491	valid_1's auc: 0.837518	valid_1's binary_logloss: 0.147104
[8]	valid_0's auc: 0.866269	valid_0's binary_logloss: 0.133803	valid_1's auc: 0.838194	valid_1's binary_logloss: 0.145883
[9]	valid_0's auc: 0.86766	valid_0's binary_logloss: 0.132376	valid_1's auc: 0.838353	valid_1's binary_logloss: 0.14489
[10]	valid_0's auc: 0.869498	valid_0's binary_logloss: 0.131036	valid_1's auc: 0.839492	valid_1's binary_logloss: 0.144015
[11]	valid_0's auc: 0.871856	valid_0's binary_logloss: 0.129821	valid_1's auc: 0.840007	valid_1's binary_logloss: 0.143293
[12]	valid_0's auc: 0.875154	valid_0's binary_logloss: 0.128677	valid_1's auc: 0.838749	valid_1's binary_logloss: 0.142749
[13]	valid_0's auc: 0.877072	valid_0's binary_logloss: 0.127675	valid_1's auc: 0.838154	valid_1's binary_logloss: 0.142217
[14]	valid_0's auc: 0.878739	valid_0's binary_logloss: 0.126748	valid_1's auc: 0.837654	valid_1's binary_logloss: 0.141853
[15]	valid_0's auc: 0

[32]	training's auc: 0.91001	training's binary_logloss: 0.114143	valid_1's auc: 0.837845	valid_1's binary_logloss: 0.139386
[33]	training's auc: 0.911283	training's binary_logloss: 0.113625	valid_1's auc: 0.838345	valid_1's binary_logloss: 0.13931
[34]	training's auc: 0.912651	training's binary_logloss: 0.113178	valid_1's auc: 0.837903	valid_1's binary_logloss: 0.139383
[35]	training's auc: 0.913931	training's binary_logloss: 0.112702	valid_1's auc: 0.838171	valid_1's binary_logloss: 0.139326
[36]	training's auc: 0.914763	training's binary_logloss: 0.1123	valid_1's auc: 0.838153	valid_1's binary_logloss: 0.139338
[37]	training's auc: 0.916018	training's binary_logloss: 0.11181	valid_1's auc: 0.83812	valid_1's binary_logloss: 0.139346
[38]	training's auc: 0.917335	training's binary_logloss: 0.111359	valid_1's auc: 0.837934	valid_1's binary_logloss: 0.139388
[39]	training's auc: 0.918504	training's binary_logloss: 0.11094	valid_1's auc: 0.837871	valid_1's binary_logloss: 0.139407
[40]	tr

In [13]:
gbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=32, sumbsample=0.8, min_child_samples=100,
                          max_depth=128)

evals = [(X_test, y_test)]
lgbm_clf.fit(X_train, y_train, early_stopping_rounds=100, eval_metric="auc", eval_set=evals,
                verbose=True)

lgbm_roc_score = roc_auc_score(y_test, lgbm_clf.predict_proba(X_test)[:,1],average='macro')
print('ROC AUC: {0:.4f}'.format(lgbm_roc_score))

[1]	valid_0's auc: 0.817877	valid_0's binary_logloss: 0.165052
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 0.817879	valid_0's binary_logloss: 0.160075
[3]	valid_0's auc: 0.82723	valid_0's binary_logloss: 0.156418
[4]	valid_0's auc: 0.831084	valid_0's binary_logloss: 0.153598
[5]	valid_0's auc: 0.833542	valid_0's binary_logloss: 0.151396
[6]	valid_0's auc: 0.833045	valid_0's binary_logloss: 0.149583
[7]	valid_0's auc: 0.834596	valid_0's binary_logloss: 0.148134
[8]	valid_0's auc: 0.836232	valid_0's binary_logloss: 0.146835
[9]	valid_0's auc: 0.837247	valid_0's binary_logloss: 0.145888
[10]	valid_0's auc: 0.838312	valid_0's binary_logloss: 0.144884
[11]	valid_0's auc: 0.839099	valid_0's binary_logloss: 0.144052
[12]	valid_0's auc: 0.83924	valid_0's binary_logloss: 0.143418
[13]	valid_0's auc: 0.83934	valid_0's binary_logloss: 0.142902
[14]	valid_0's auc: 0.839012	valid_0's binary_logloss: 0.142465
[15]	valid_0's auc: 0.839228	valid_0's binary_logloss: